# Demo Boskalis

## Create Graph from Project Documents

In [33]:
import pandas as pd
import numpy as np
import os
from dotenv import load_dotenv
from pathlib import Path
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from pydantic import BaseModel, Field, validator
from typing import List, Optional
from openai import OpenAI
import time
import json
from json import loads, dumps
from neo4j import Query, GraphDatabase, RoutingControl, Result
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from IPython.display import clear_output
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin

In [ ]:
env_file = '../.env'

In [3]:
if os.path.exists(env_file):
    load_dotenv(env_file, override=True)

    # Neo4j
    HOST = os.getenv('NEO4J_URI')
    USERNAME = os.getenv('NEO4J_USERNAME')
    PASSWORD = os.getenv('NEO4J_PASSWORD')
    DATABASE = os.getenv('NEO4J_DATABASE')

    # AI
    OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')
    os.environ['OPENAI_API_KEY']=OPENAI_API_KEY
    LLM = os.getenv('LLM')
    EMBEDDINGS_MODEL = os.getenv('EMBEDDINGS_MODEL')
else:
    print(f"File {env_file} not found.")

In [4]:
projects_documents_path = "documents/projects/"
equipment_documents_path = "documents/equipment/"

### Test Connection to Neo4j

In [5]:
driver = GraphDatabase.driver(
    HOST,
    auth=(USERNAME, PASSWORD)
)

In [6]:
driver.execute_query(
    """
    MATCH (n) RETURN COUNT(n) as Count
    """,
    database_=DATABASE,
    routing_=RoutingControl.READ,
    result_transformer_= lambda r: r.to_df()
)

,Count
0,0


## Chunk and Load Documents

In [7]:
chunk_size = 300
chunk_overlap = 50

In [8]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = chunk_size,
    chunk_overlap  = chunk_overlap,
    length_function = len,
    is_separator_regex = False,
)

In [9]:
def parse_documents(documents_path):
    documents_dir = Path(documents_path)
    chunks_with_metadata = []

    for pdf_path in documents_dir.glob("*.pdf"):
        print(f"Parsing: {pdf_path.name}")
        loader = PyPDFLoader(str(pdf_path))
        pages = loader.load_and_split()
        total_chunks_for_doc = 0
        for page in pages:
            page_chunks = text_splitter.split_text(page.page_content)
            for chunk in page_chunks:
                chunks_with_metadata.append({
                    "doc_path": page.metadata.get("doc_path"),
                    "doc_name": pdf_path.name,
                    "page": page.metadata.get("page"),
                    "text": chunk,
                    "num_chunks": len(page_chunks)
                })
                total_chunks_for_doc += 1
        print(f"Chunked {len(pages)} pages into {total_chunks_for_doc} chunks")
    return pd.DataFrame.from_dict(chunks_with_metadata)

In [10]:
project_chunks_df = parse_documents(projects_documents_path)
equipment_chunks_df = parse_documents(equipment_documents_path)

Parsing: italy_-_genoa.pdf
Chunked 2 pages into 21 chunks
Parsing: FPSo Transportation.pdf
Chunked 2 pages into 17 chunks
Parsing: Haven Jack Up Upgrade.pdf
Chunked 2 pages into 17 chunks
Parsing: fox_river_leaflet.pdf
Chunked 1 pages into 6 chunks
Parsing: uk_-_peterhead.pdf
Chunked 2 pages into 19 chunks
Parsing: egypt_-suez_canal.pdf
Chunked 2 pages into 25 chunks
Parsing: togo-benin.pdf
Chunked 3 pages into 36 chunks
Parsing: Gulhifalhu Phase 2 Maldives.pdf
Chunked 2 pages into 31 chunks
Parsing: spain_-_seville.pdf
Chunked 2 pages into 23 chunks
Parsing: abu_dhabi_-_khalifa.pdf
Chunked 4 pages into 41 chunks
Parsing: 2602020_borssele_alpha.pdf
Chunked 2 pages into 22 chunks
Parsing: def_baltic_ii.pdf
Chunked 2 pages into 25 chunks
Parsing: suriname-river.pdf
Chunked 2 pages into 26 chunks
Parsing: the_netherlands_-_4_nederrijn_measures_room_for_the_river.pdf
Chunked 2 pages into 21 chunks
Parsing: a2_holendrecht-maarsen.pdf
Chunked 2 pages into 23 chunks
Parsing: netherlands-north

In [11]:
project_chunks_df['category'] = 'project'
equipment_chunks_df['category'] = 'equipment'

In [12]:
chunks_df = pd.concat([project_chunks_df, equipment_chunks_df], ignore_index=True)
chunks_df['chunk_seq_id'] = chunks_df.index + 1

In [13]:
chunks_df

,source,doc_name,page,text,num_chunks,category,chunk_seq_id
0,documents/projects/italy_-_genoa.pdf,italy_-_genoa.pdf,0,INTRODUCTION\nThe deepening and extension of t...,11,project,1
1,documents/projects/italy_-_genoa.pdf,italy_-_genoa.pdf,0,"a joint venture with Tecnis S.p.A., an Italian...",11,project,2
2,documents/projects/italy_-_genoa.pdf,italy_-_genoa.pdf,0,Genoa. \nSCOPE \nThe work has been split up in...,11,project,3
3,documents/projects/italy_-_genoa.pdf,italy_-_genoa.pdf,0,The Trailing Suction Hopper Dredger (TSHD) Ast...,11,project,4
4,documents/projects/italy_-_genoa.pdf,italy_-_genoa.pdf,0,"B1, work continues on filling up area A1, \nFe...",11,project,5
...,...,...,...,...,...,...,...
857,documents/equipment/shoreway.pdf,shoreway.pdf,0,Max. draught Int. load line 5.70 m\nMax. drau...,7,equipment,858
858,documents/equipment/shoreway.pdf,shoreway.pdf,0,Sailing speed loaded 13.0 kn\nTotal installe...,7,equipment,859
859,documents/equipment/shoreway.pdf,shoreway.pdf,0,Bow thruster 450 kW\nEQUIPMENT\nSHEET\nSHORE...,7,equipment,860
860,documents/equipment/shoreway.pdf,shoreway.pdf,1,The information contained in this data sheet i...,2,equipment,861


### Load Documents

In [14]:
document_df = (
    chunks_df.groupby("doc_name").agg(
        doc_path=("doc_path", "max"),
        full_text=("text", " ".join),
        pages=("page", "max"),
        chunks=("num_chunks", "max"),
        category=("category", "max")
    )
    .reset_index()
)
document_df['pages'] = document_df['pages'].apply(lambda x: x + 1)

In [15]:
document_df

,doc_name,source,full_text,pages,chunks,category
0,20181127_aberdeen_bay_offshore_windfarm_projec...,documents/projects/20181127_aberdeen_bay_offsh...,A At the fabrication Yard in Newcastle\nA\nPRO...,2,16,project
1,20201115_def_wintershall_j6-a.pdf,documents/projects/20201115_def_wintershall_j6...,PROJECT\nSHEET\nUMBILICAL REPLACEMENT AND \nA...,2,11,project
2,2602020_borssele_alpha.pdf,documents/projects/2602020_borssele_alpha.pdf,A Schematic of cable route with in yellow the ...,2,13,project
3,FPSo Transportation.pdf,documents/projects/FPSo Transportation.pdf,FEATURES\nClient COOEC (Offshore Oil Engineeri...,2,10,project
4,Gulhifalhu Phase 2 Maldives.pdf,documents/projects/Gulhifalhu Phase 2 Maldives...,A Gulhifalhu south east corner\nB Revetment in...,2,16,project
5,Harbor of Lerwick.pdf,documents/projects/Harbor of Lerwick.pdf,INTRODUCTION \nThe Port of Lerwick is situated...,2,15,project
6,Haven Jack Up Upgrade.pdf,documents/projects/Haven Jack Up Upgrade.pdf,FEATURES\nClient Lamprell\nCompany Master Mari...,2,9,project
7,Project Sheet LNG Canada Mar 24.pdf,documents/projects/Project Sheet LNG Canada Ma...,A Overview of project site\nB Trailing suction...,2,16,project
8,a2_holendrecht-maarsen.pdf,documents/projects/a2_holendrecht-maarsen.pdf,"PROJECT \nSHEET\nHOLENDRECHT - MAARSSEN, THE N...",2,14,project
9,abu_dhabi_-_khalifa.pdf,documents/projects/abu_dhabi_-_khalifa.pdf,INTRODUCTION \nThe Khalifa Port and Industrial...,4,15,project


Load URL's of Documents

In [16]:
PAGE = "https://boskalis.com/download-center"
BASE = "https://boskalis.com"

In [17]:
def fetch_pdf_table():
    r = requests.get(PAGE, headers={"User-Agent": "Mozilla/5.0"}, timeout=20)
    r.raise_for_status()
    soup = BeautifulSoup(r.text, "html.parser")

    rows = []
    seen = set()

    for a in soup.find_all("a", href=True):
        href = a["href"].split("?")[0].strip()
        if href.lower().endswith(".pdf"):
            full = href if href.startswith("http") else urljoin(BASE, href)
            if full in seen:
                continue
            seen.add(full)
            file_name = full.rstrip("/").split("/")[-1]  # keep .pdf
            rows.append({"doc_name": file_name, "url": full})

    df = pd.DataFrame(rows).drop_duplicates().reset_index(drop=True)
    return df

In [18]:
urls_df = fetch_pdf_table()

In [19]:
document_df = pd.merge(document_df, urls_df, on="doc_name", how="left")

In [20]:
merge_document_query = """
    MERGE(mergedDocument:Document {name: $doc_name})
    SET mergedDocument.doc_path = $doc_path,
        mergedDocument.full_text = $full_text,
        mergedDocument.pages = $pages,
        mergedDocument.chunks = $chunks,
        mergedDocument.category = $category,
        mergedDocument.url = $url
        
    RETURN mergedDocument
    """

In [21]:
for index, row in document_df.iterrows():
    clear_output(wait=True)
    driver.execute_query(
        merge_document_query,
        database_=DATABASE,
        routing_=RoutingControl.WRITE,
        doc_name = row.doc_name,
        doc_path = row.doc_path,
        full_text = row.full_text,
        pages = row.pages,
        chunks = row.chunks,
        category = row.category,
        url = row.url,
    )
    print(f"Loaded {row['doc_name']}")
    print("Progress: ", np.round((index+1)/document_df.shape[0]*100,2), "%")

Loaded willem-van-oranje.pdf
Progress:  100.0 %


### Load Chunks

First add Embeddings

In [22]:
embeddings_model = OpenAIEmbeddings(
    model = EMBEDDINGS_MODEL,
    openai_api_key = OPENAI_API_KEY
)

Add an embedding for every chunk in the DataFrame

In [23]:
chunks_df['embedding'] = chunks_df['text'].apply(lambda x: embeddings_model.embed_query(x))

In [24]:
chunks_df.head()

,source,doc_name,page,text,num_chunks,category,chunk_seq_id,embedding
0,documents/projects/italy_-_genoa.pdf,italy_-_genoa.pdf,0,INTRODUCTION\nThe deepening and extension of t...,11,project,1,"[0.006220303475856781, -0.0248403362929821, 0...."
1,documents/projects/italy_-_genoa.pdf,italy_-_genoa.pdf,0,"a joint venture with Tecnis S.p.A., an Italian...",11,project,2,"[0.007828463800251484, -0.025723079219460487, ..."
2,documents/projects/italy_-_genoa.pdf,italy_-_genoa.pdf,0,Genoa. \nSCOPE \nThe work has been split up in...,11,project,3,"[-0.003091271501034498, -0.02294781804084778, ..."
3,documents/projects/italy_-_genoa.pdf,italy_-_genoa.pdf,0,The Trailing Suction Hopper Dredger (TSHD) Ast...,11,project,4,"[-0.006104578264057636, -0.005901559256017208,..."
4,documents/projects/italy_-_genoa.pdf,italy_-_genoa.pdf,0,"B1, work continues on filling up area A1, \nFe...",11,project,5,"[-0.0036728905979543924, -0.03611675649881363,..."


In [25]:
merge_chunck_query = """
    MERGE(c:Chunk {id: $chunk_seq_id})
        ON CREATE SET
            c.page = $page,
            c.text = $text,
            c.embedding = $embedding
    WITH c
    MATCH (d:Document{name: $doc_name})
    MERGE (c)-[r:PART_OF]->(d)
"""

In [26]:
for index, row in chunks_df.iterrows():
    clear_output(wait=True)
    driver.execute_query(
        merge_chunck_query,
        database_=DATABASE,
        routing_=RoutingControl.WRITE,
        chunk_seq_id = row.chunk_seq_id,
        page = row.page,
        text = row.text,
        embedding = row.embedding,
        doc_name = row.doc_name
    )
    print("Progress: ", np.round(((index+1)/chunks_df.shape[0])*100,2), "%")

Progress:  100.0 %


## Project Entities

Let's extract projects from a Project Documents

### Extract Project Entities

In [27]:
class Location(BaseModel):
    """
    Represents the location of a project
    """
    location: str = Field(..., description="Location were the project was carried out")
    country: str =  Field(..., description="Country where the location is based")
    country_code: str = Field(..., description="Country code according to the ISO 3166-1 alpha-3 country codes")

class Equipment(BaseModel):
    """
    Represents the Equipment used in the project. This class contains one equipment with its specific name and no generic name.
    """
    name: str = Field(..., description="Name of the equipment used in the project")
    description: Optional[str] =  Field(..., description="Short description of the equipment useded in the project if provided")

class Contractor(BaseModel):
    """
    Represents a Contractor of a project
    """
    name: str = Field(..., description="Name of the contractor of the project")
    description: Optional[str] =  Field(..., description="Short description of the contractor if provided")

class Client(BaseModel):
    """
    Represents a client of a project
    """
    name: str = Field(..., description="Name of the client of the project")
    description: Optional[str] =  Field(..., description="Short description of the client if provided")

class Project(BaseModel):
    """
    Represents a question with answer from the press conferences with ECB.
    """
    doc_path: str = Field(..., description="The path of the document")
    name: str = Field(..., description="The name of the project")
    date: str = Field(..., description="The date of the project")
    description: str = Field(..., description="Short description of the project that was carried out")
    client: List[Client] = Field(..., description="List of clients for this project")
    contractor: List[Contractor] = Field(..., description="List of contractors for this project")
    equipment: List[Equipment] = Field(..., description="List of equipment used in this project. If multiple equipment is used split into different objects.")
    location: List[Location] = Field(..., description="Location of this project")

class ProjectList(BaseModel):
    """
    Represents a list of projects based carried out. 
    """
    projects: List[Project]

In [28]:
system_message = """
    You are an expert in extracting structured information text. 
    In this case we want to extract Information of projects from documents. The text is coming from short documents on a project carried out by Boskalis.
    Please extract the information as specified in the provided format.

    Identify details such as:
    - Project that is carried out.
    - More information like equipment used, contractor, client and location the project was carried out.
    - Please be specific on the contractor. Typically these are different entities of Boskalis. 
    - If a description is asked, please generate a short summary based on the text.
    - For Equipment use the specific name of that equipment no generic names as (e.g. ignore Grab dredger, Suction Hopper Dredger, etc). 

    Don't come up with anything yourself. Just base it on the documents fully.
"""

In [29]:
client = OpenAI()

In [30]:
def extract(document, model=LLM, temperature=0):
    response = client.beta.chat.completions.parse(
        model=model,
        temperature=temperature,
        messages=[
            {"role": "system", "content": system_message},
            {"role": "user", "content": document},
        ],
        response_format=ProjectList,
    )
    return json.loads(response.choices[0].message.content)

In [31]:
columns = ['doc_path', 'name', 'date', 'description', 'client', 'contractor', 'equipment', 'location']
projects_df = pd.DataFrame(columns=columns)

In [34]:
project_documents_df = document_df[document_df['category'] == 'project'].reset_index(drop=True)

for idx, row in project_documents_df.iterrows():
    text = project_documents_df['full_text'].iloc[idx]
    extracted_projects = extract("Document Path:" + project_documents_df['doc_path'].iloc[idx] + "\n" + text)
    ProjectList.model_validate(extracted_projects)
    df = pd.json_normalize(extracted_projects["projects"])
    projects_df = pd.concat([projects_df, df], ignore_index=True)

In [35]:
projects_df

,source,name,date,description,client,contractor,equipment,location
0,documents/projects/20181127_aberdeen_bay_offsh...,Aberdeen Bay Offshore Windfarm Project,2016 - 2018,"The Aberdeen Bay Offshore Windfarm Project, al...","[{'name': 'Vattenfall', 'description': None}]","[{'name': 'Boskalis', 'description': 'Royal Bo...","[{'name': 'Asian Hercules III', 'description':...","[{'location': 'Aberdeen, Scotland', 'country':..."
1,documents/projects/20201115_def_wintershall_j6...,Umbilical Replacement and Accumulator Rack Cha...,"Q2, 2019",The project involved replacing the SSIV contro...,"[{'name': 'Wintershall Noordzee B.V.', 'descri...","[{'name': 'Boskalis', 'description': 'Royal Bo...","[{'name': 'DSV ‘EDT Protea’', 'description': N...","[{'location': 'Block J6-A', 'country': 'The Ne..."
2,documents/projects/2602020_borssele_alpha.pdf,Borssele Alpha/Beta Export Cables,2018 - 2020,The project involved the supply and installati...,"[{'name': 'Tennet TSO B.V.', 'description': No...",[{'name': 'VBNK (consortium Boskalis Subsea Ca...,"[{'name': 'Cable Lay Barge Giant 7', 'descript...","[{'location': 'Borssele', 'country': 'Netherla..."
3,documents/projects/FPSo Transportation.pdf,P67 FPSO Transportation,2018,Boskalis executed the dry transportation of th...,[{'name': 'COOEC (Offshore Oil Engineering Co....,"[{'name': 'Royal Boskalis Westminster', 'descr...","[{'name': 'MV Boka Vanguard', 'description': '...","[{'location': 'Shanghai', 'country': 'China', ..."
4,documents/projects/Gulhifalhu Phase 2 Maldives...,Gulhifalhu Phase 2,2023 - 2024,The Gulhifalhu Phase 2 project involved the ex...,[{'name': 'Government of the Republic of Maldi...,[{'name': 'Boskalis Westminster Contracting Li...,"[{'name': 'Oranje', 'description': 'A trailing...","[{'location': 'Gulhifalhu Lagoon, North Malé A..."
5,documents/projects/Harbor of Lerwick.pdf,North Harbor of Lerwick Dredging and Reclamation,2007 – 2008,The project involved dredging and reclamation ...,"[{'name': 'Lerwick Port Authority', 'descripti...","[{'name': 'Westminster Dredging Co.', 'descrip...","[{'name': 'TSHD Waterway', 'description': 'Use...","[{'location': 'Port of Lerwick, Scotland', 'co..."
6,documents/projects/Haven Jack Up Upgrade.pdf,Haven Jack-Up Upgrade,January - February 2018,The Haven Jack-Up Upgrade project involved the...,"[{'name': 'Lamprell', 'description': None}]","[{'name': 'Royal Boskalis Westminster', 'descr...","[{'name': 'Asian Hercules III', 'description':...","[{'location': 'CCB Ågotnes', 'country': 'Norwa..."
7,documents/projects/Project Sheet LNG Canada Ma...,LNG Canada Dredging and Disposal Services,2018 - 2021,Boskalis was involved in the LNG Canada projec...,"[{'name': 'LNG Canada Development Inc., Shell'...","[{'name': 'Boskalis', 'description': 'Boskalis...","[{'name': 'Beachway', 'description': 'Trailing...","[{'location': 'Kitimat, British Columbia', 'co..."
8,documents/projects/a2_holendrecht-maarsen.pdf,Reconstruction of A2 Motorway Holendrecht-Maar...,October 2006 – mid-2012,The project involved the reconstruction and wi...,"[{'name': 'Rijkswaterstaat', 'description': 'D...","[{'name': 'A2HoMa consortium', 'description': ...","[{'name': 'Hydronamic', 'description': 'Boskal...","[{'location': 'A2 Motorway, Utrecht – Amsterda..."
9,documents/projects/abu_dhabi_-_khalifa.pdf,Khalifa Port and Industrial Zone (KPIZ) Project,October 2007 – January 2012,The Khalifa Port and Industrial Zone (KPIZ) pr...,"[{'name': 'Abu Dhabi Ports Company (ADPC)', 'd...",[{'name': 'Khalifa Port Marine Consortium (KPM...,"[{'name': 'TsHD Alpha B', 'description': 'A sh...","[{'location': 'Al Taweelah, Abu Dhabi', 'count..."


### Load Projects to Neo4j

In [38]:
CREATE_PROJECT_QUERY = """
MERGE (p:Project {name: $name})
SET  p.doc_path = $doc_path,
     p.date = $date,
     p.description = $description

WITH p
CALL (p) {
    MATCH (d:Document{doc_path: $doc_path})
    MERGE (p)-[:HAS_DOCUMENT]->(d)
}

WITH p
CALL (p) {
    UNWIND $clients AS c
    MERGE (client:Client {name: c.name})
    SET client.description = c.description
    MERGE (p)-[:HAS_CLIENT]->(client)
}

WITH p
CALL (p) {
    UNWIND $contractors AS ct
    MERGE (contractor:Contractor {name: ct.name})
    SET contractor.description = ct.description
    MERGE (p)-[:HAS_CONTRACTOR]->(contractor)
}

WITH p
CALL (p) {
    UNWIND $equipment AS e
    MERGE (eq:Equipment {name: e.name})
    SET eq.description = e.description
    MERGE (p)-[:USED_EQUIPMENT]->(eq)
}

WITH p
CALL (p) {
    UNWIND $locations AS l
    MERGE (loc:Location {name: l.location})
    MERGE (country:Country {code: l.country_code})
    SET country.name = l.country
    MERGE (loc)-[:IN_COUNTRY]->(country)
    MERGE (p)-[:AT_LOCATION]->(loc)
};
"""

In [39]:
def insert_project_row(row):
    driver.execute_query(
        CREATE_PROJECT_QUERY,
        name=row["name"],
        doc_path=row["doc_path"],
        date=row["date"],
        description=row["description"],
        clients=row["client"],
        contractors=row["contractor"],
        equipment=row["equipment"],
        locations=row["location"],     
        database_=DATABASE,
        routing_=RoutingControl.WRITE,
    )

In [40]:
for idx, row in projects_df.iterrows():
    insert_project_row(row)

In [41]:
driver.execute_query(
    """
    MATCH (p:Project) RETURN COUNT(p) AS project_count
    """,
    database_=DATABASE,
    routing_=RoutingControl.READ,
    result_transformer_=lambda r: r.to_df()
)

,project_count
0,26


## Equipment Entities

Let's extract projects from a Equipment Documents

### Extract Equipment Entities

In [42]:
class Location(BaseModel):
    """
    Represents the location of equipment
    """
    location: str = Field(..., description="Location were the equipment was registered")
    country: str =  Field(..., description="Country where the location is based")
    country_code: str = Field(..., description="Country code according to the ISO 3166-1 alpha-3 country codes")

class Equipment(BaseModel):
    """
    Represents the equipment that can be used in projects
    """
    doc_path: str = Field(..., description="The path of the document")
    name: str = Field(..., description="Name of the equipment used in the project")
    year_of_construction: Optional[int] =  Field(..., description="Year of the equipment being constructed")
    capacity: Optional[str] =  Field(..., description="Capacity of the equipment")
    built_by: Optional[str] =  Field(..., description="Company the equipment was being built by")
    imo_number: Optional[int] =  Field(..., description="The IMO (International Maritime Organization)")
    port_of_registery: List[Location] = Field(..., description="Location of the port of registery of this equipment")
    description: Optional[str] =  Field(..., description="Short description of the equipment useded in the project if provided")

class EquipmentList(BaseModel):
    """
    Represents a list of equipment described in the documents. 
    """
    equipments: List[Equipment]

In [43]:
system_message = """
    You are an expert in extracting structured information text. 
    In this case we want to extract information of equipment from documents. The text is coming from short documents on equipment of Boskalis.
    Please extract the information as specified in the provided format.

    Identify details such as:
    - Equipment that is described.
    - More properties like description, capacity, year of construction, classification etc. 

    Don't come up with anything yourself. Just base it on the documents fully.
"""

In [44]:
client = OpenAI()

In [45]:
def extract(document, model=LLM, temperature=0):
    response = client.beta.chat.completions.parse(
        model=model,
        temperature=temperature,
        messages=[
            {"role": "system", "content": system_message},
            {"role": "user", "content": document},
        ],
        response_format=EquipmentList,
    )
    return json.loads(response.choices[0].message.content)

In [46]:
columns = ['doc_path', 'name', 'year_of_construction', 'capacity', 'built_by', 'imo_number', 'port_of_registery', 'description']
equipment_df = pd.DataFrame(columns=columns)

In [47]:
equipment_documents_df = document_df[document_df['category'] == 'equipment'].reset_index(drop=True)

for idx, row in equipment_documents_df.iterrows():
    text = equipment_documents_df['full_text'].iloc[idx]
    extracted_equipments = extract("Document Path:" + equipment_documents_df['doc_path'].iloc[idx] + "\n" + text)
    EquipmentList.model_validate(extracted_equipments)
    df = pd.json_normalize(extracted_equipments["equipments"])
    equipment_df = pd.concat([equipment_df, df], ignore_index=True)

In [48]:
equipment_df

,source,name,year_of_construction,capacity,built_by,imo_number,port_of_registery,description
0,documents/equipment/alexii.pdf,ALEX II,2006,"Gross tonnage 548 GT, Net tonnage 164 NT, Leng...",Yizheng Changrun Shipyard,None,[],"Floating grab dredger with no propulsion, unre..."
1,documents/equipment/asian-hercules-iii.pdf,Asian Hercules III,2015,"5,000 tons floating sheerleg",None,None,"[{'location': 'Singapore', 'country': 'Singapo...",A heavy lift vessel with a floating sheerleg c...
2,documents/equipment/bear-27-03.pdf,BEAR,1999,"Bunker capacity fuel 1545 m3, fresh water 522 ...",None,9185932,"[{'location': 'Antwerp', 'country': 'Belgium',...",Offshore vessel used for anchor handling tug s...
3,documents/equipment/boka-vanguard.pdf,BOKA VANGUARD,2012,"Deadweight 116,175 t / 130,000 t",Hyundai Heavy Industries,9618783,"[{'location': 'Willemstad', 'country': 'Curaça...",Semi-submersible heavy transport vessel
4,documents/equipment/boxer.pdf,BOXER,2010,"Bunker capacity 378 cbm, Fuel: 378 m³, Fresh w...",None,9537537,"[{'location': 'Valletta', 'country': 'Malta', ...",Offshore vessel / Anchor handling tug (AHT) wi...
5,documents/equipment/bssiii.pdf,BSSIII Burial Sledge System,None,"Total installed jetting power 1,800 kW",None,None,[],"Boskalis has developed the burial tool BSSIII,..."
6,documents/equipment/cornelius-backhoe.pdf,Cornelius Backhoe Dredger,1999,None,De Donge / Terramarre,None,[],Special service / Backhoe dredger - no propuls...
7,documents/equipment/crestway.pdf,Crestway,2008,"Carrying capacity (D.W.) 8,350 t, Hopper capac...",IHC Dredgers B.V.,None,[],Trailing suction hopper dredger with mechanica...
8,documents/equipment/edax.pdf,EDAX,1985,None,Merwede Shipyard B.V.,None,[],Cutter Suction Dredger
9,documents/equipment/giant-7.pdf,GIANT 7,2015,None,"Keppel Nantong Shipyard Co., Ltd.",None,"[{'location': 'Belgium', 'country': 'Belgium',...",GIANT 7 is a cable lay barge with a classifica...


### Load Equipment to Neo4j

In [49]:
CREATE_EQUIPMENT_QUERY = """
    MERGE (e:Equipment {name: $name})
    SET  e.doc_path = $doc_path,
         e.year_of_construction = $year_of_construction,
         e.capacity = $capacity,
         e.imo_number = $imo_number,
         e.description = $description
    
    WITH e
    CALL (e) {
        MATCH (d:Document{doc_path: $doc_path})
        MERGE (e)-[:HAS_DOCUMENT]->(d)
    }
    
    WITH e
    CALL (e) {
        UNWIND $port_of_registery AS l
        MERGE (loc:Location {name: l.location})
        MERGE (country:Country {code: l.country_code})
        SET country.name = l.country
        MERGE (loc)-[:IN_COUNTRY]->(country)
        MERGE (e)-[:REGISTERED_AT]->(loc)
    }
    ;
"""

In [50]:
def insert_equipment_row(row):
    driver.execute_query(
        CREATE_EQUIPMENT_QUERY,
        name=row["name"],
        doc_path=row["doc_path"],
        year_of_construction=row["year_of_construction"],
        capacity=row["capacity"],
        built_by=row["built_by"],
        imo_number=row["imo_number"],
        description=row["description"],
        port_of_registery=row["port_of_registery"],
        database_=DATABASE,
        routing_=RoutingControl.WRITE,
    )


In [51]:
for idx, row in equipment_df.iterrows():
    insert_equipment_row(row)

## Resolve duplicate Entities

Refactor locations in countries

In [52]:
df = driver.execute_query(
    """
        MATCH ()-[r1]-(l:Location)-[r2:IN_COUNTRY]-(c:Country)
        WHERE l.name = c.name
        DELETE r2
        WITH r1, c
        CALL apoc.refactor.to(r1, c) YIELD input, output
        RETURN input, output
    """,
    database_=DATABASE,
    routing_=RoutingControl.WRITE,
    result_transformer_=lambda r: r.to_df()
)

Refactor equipment names

In [53]:
merged_df = driver.execute_query(
    """
        MATCH (e1:Equipment)
        WHERE e1.doc_path IS NOT NULL
        CALL (e1) {
         MATCH (e2:Equipment)
         WITH e1, e2, 
         apoc.text.join(apoc.coll.sort(split(trim( apoc.text.replace(apoc.text.replace(lower(e1.name), "[^a-z0-9 ]", " "), "\\s+", " ")), " ")), " ") as normalized_1,
         apoc.text.join(apoc.coll.sort(split(trim( apoc.text.replace(apoc.text.replace(lower(e2.name), "[^a-z0-9 ]", " "), "\\s+", " ")), " ")), " ") as normalized_2
         WHERE e1 <> e2 AND (LOWER(replace(normalized_2, " ", "")) CONTAINS LOWER(replace(normalized_1, " ", "")) OR (apoc.text.replace(lower(e2.name), "[^a-z0-9 ]", " ") CONTAINS apoc.text.replace(lower(e1.name), "[^a-z0-9 ]", " ")))
         WITH e1, e2 
         OPTIONAL MATCH (:Project)-[r]->(e2)
         CALL apoc.refactor.to(r, e1) YIELD input, output
         WITH COLLECT(e2.name) as similar_names
         RETURN similar_names
        }
        WITH e1, similar_names
        CALL (similar_names) {
          UNWIND similar_names as name
          MATCH (e:Equipment{name: name})
          DELETE e
        }
        WITH e1, similar_names
        RETURN e1.name, similar_names
    """,
    database_=DATABASE,
    routing_=RoutingControl.WRITE,
    result_transformer_=lambda r: r.to_df()
)
merged_df

,e1.name,similar_names
0,Asian Hercules III,[]
1,Willem van Oranje,"[TSHD Willem van Oranje, TSHD Willem van Oranje]"
2,Waterway,[TSHD Waterway]
3,ALEX II,[Alex II]
4,BEAR,[Union Bear]
5,BOKA VANGUARD,[MV Boka Vanguard]
6,BOXER,[Union Boxer]
7,BSSIII Burial Sledge System,[]
8,Cornelius Backhoe Dredger,[]
9,Crestway,[TSHD Crestway]


In [54]:
equipment_names_df = driver.execute_query(
    """
        MATCH (e1:Equipment)
        WHERE e1.doc_path IS NOT NULL
        RETURN e1.name as equipment_name
    """,
    database_=DATABASE,
    routing_=RoutingControl.READ,
    result_transformer_=lambda r: r.to_df()
)
equipment_names = list(equipment_names_df['equipment_name'])

In [55]:
equipment_names_df = driver.execute_query(
    """
        MATCH (e1:Equipment)
        WHERE e1.doc_path IS NULL
        RETURN e1.name as equipment_name
    """,
    database_=DATABASE,
    routing_=RoutingControl.READ,
    result_transformer_=lambda r: r.to_df()
)
parsed_names = list(equipment_names_df['equipment_name'])

In [56]:
def detect_duplicates(equipment_names, parsed_names):
    prompt = f"""
        You are an entity resolution assistant.
        Given the following list of equipment names:
        equipment_names:
        {equipment_names}

        can you find any equipment names in the following list that describe the same equipment?
        potential_duplicates:
        {parsed_names}
        
        Return ONLY a JSON object with the following structure:
        {{
            "groups": [
                {{
                    "canonical": "<name>",
                    "duplicates": ["duplicate1", "duplicate2", ...]
                }},
                ...
            ]
        }}
        
        Rules:
        - Only include TRUE duplicates (different spellings or formatting of the same equipment).
        - Do NOT include generic categories (e.g., "TSHDs"). 
        - Please focus on equipment names instead of generic descriptions (Grab dredger can be ignored). 
        - If the duplicates list is empty, don't return the group. Only return groups with duplicates.
        - Only return the JSON object no prefix or whatsoever. 
        - The canonical should always come from the equipment name. The duplicates can only be coming from the potential_duplicates.
    """

    response = client.chat.completions.create(
        model=LLM,
        messages=[{"role": "user", "content": prompt}]
    )
    content = response.choices[0].message.content.strip()[8:-4]
    try:
        data = json.loads(content)
    except json.JSONDecodeError:
        raise ValueError("LLM returned invalid JSON:\n" + content)
    return data

In [57]:
data = detect_duplicates(equipment_names, parsed_names)

In [58]:
data

{'groups': [{'canonical': 'BSSIII Burial Sledge System',
   'duplicates': ['Burial Sledge BSS-III']},
  {'canonical': 'Cornelius Backhoe Dredger', 'duplicates': ['Cornelius']},
  {'canonical': 'Willem van Oranje',
   'duplicates': ['Oranje', 'TSHD ‘Oranje’']}]}

In [59]:
def merge_duplicate_group(canonical, duplicates):
    print(f"Merging duplicates into: {canonical}")

    driver.execute_query(
        """
            MATCH (c:Equipment {name: $canonical})
            WHERE c.doc_path IS NOT NULL
            WITH c
            UNWIND $dups AS dupName
                MATCH (d:Equipment {name: dupName})
                OPTIONAL MATCH (p)-[r]->(d)
                CALL apoc.refactor.to(r, c) YIELD input, output
                DETACH DELETE d
            RETURN c.name AS kept
        """,
        database_=DATABASE,
        canonical=canonical,
        dups=duplicates,
        routing_=RoutingControl.WRITE,
        result_transformer_=lambda r: r.to_df()
    )

In [60]:
data['groups']

[{'canonical': 'BSSIII Burial Sledge System',
  'duplicates': ['Burial Sledge BSS-III']},
 {'canonical': 'Cornelius Backhoe Dredger', 'duplicates': ['Cornelius']},
 {'canonical': 'Willem van Oranje', 'duplicates': ['Oranje', 'TSHD ‘Oranje’']}]

In [61]:
for group in data['groups']:
    canonical = group["canonical"]
    duplicates = group["duplicates"]
    merge_duplicate_group(canonical, duplicates)

Merging duplicates into: BSSIII Burial Sledge System
Merging duplicates into: Cornelius Backhoe Dredger
Merging duplicates into: Willem van Oranje


In [62]:
driver.execute_query(
    """
    MATCH (a)-[r:USED_EQUIPMENT]->(b)
    WITH a, b, collect(r) AS rels
    WHERE size(rels) > 1
    FOREACH (r IN rels[1..] | DELETE r)
    """,
    database_=DATABASE,
    routing_=RoutingControl.WRITE,
    result_transformer_=lambda r: r.to_df()
)

""


### Delete doc_path from Project/Equipment

Now that we have relations to documents we can delete the doc_path from Project and Equipment and give the URL instead.

In [ ]:
driver.execute_query(
    """
    MATCH (p:Project|Equipment)
    SET p.doc_path = NULL
    """,
    database_=DATABASE,
    routing_=RoutingControl.WRITE,
    result_transformer_=lambda r: r.to_df()
)

In [1]:
driver.execute_query(
    """
    MATCH (p:Project|Equipment)-[]-(d:Document)
    SET p.url = d.url
    """,
    database_=DATABASE,
    routing_=RoutingControl.WRITE,
    result_transformer_=lambda r: r.to_df()
)

NameError: name 'driver' is not defined